# Lambda=4, lambda =0.5: EvolvedOperatorAnsatz

In [1]:
import numpy as np
import pandas as pd
import pylab
import matplotlib.pyplot as plt
import time

import qiskit
from qiskit.quantum_info import Pauli
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import NumPyEigensolver
from qiskit.circuit.library import EvolvedOperatorAnsatz

import sys
sys.path.append('../utility')
from vqe_run import *
from qc_ansatze import *

In [2]:
def open_n_process(file_link):
    file = open(file_link, "r")
    content = file.read()

    data = content.split(',\n')
    data[0] = data[0].split('\n')[1]
    data[-1] = data[-1].split('\n')[0]

    data_coeff = []
    data_str = []
    for i in range(len(data)):
        data_coeff.append(float(data[i].split('*')[0]))
        data_str.append(data[i].split('*')[1].split(' ')[1])
    return data_coeff, data_str
    
def check_min_max(data_coeff):
    print(f'Min absolute value is {np.round(np.min(np.abs(data_coeff[1:])),6)}')
    print(f'Max absolute value is {np.round(np.max(np.abs(data_coeff[1:])),5)}')
    print(f'Mean absolute value is {np.round(np.mean(np.abs(data_coeff[1:])),5)}')
    
def get_k_largest_ops(data_coeff, data_str, k):
    ind_list = np.argsort(np.abs(data_coeff[1:]))[-k:]
    ops = [data_str[1:][i] for i in ind_list]
    val= [np.round(data_coeff[1:][i],4) for i in ind_list]
    return ind_list, ops, val

# Load H

In [3]:
data_coeff, data_str = open_n_process("../utility/pauliH_L4_g0.5.txt")
Hpauli = list(zip(data_str, data_coeff))
H4q = SparsePauliOp.from_list(Hpauli)
# exactly diagonalize the system using numpy routines
solver = NumPyEigensolver(k=4)
exact_solution = solver.compute_eigenvalues(H4q)
#print("Exact Result of qubit hamiltonian:", np.real(exact_solution.eigenvalues))
E_exact = np.round(np.real(exact_solution.eigenvalues)[0],5)
print(f'{check_min_max(data_coeff)}')
print(f'E_exact = {E_exact}')

Min absolute value is 0.002244
Max absolute value is 1.375
Mean absolute value is 0.07067
None
E_exact = 3.29894


In [4]:
#get the indices of the N largest elements
ind15, ops15, vals15 = get_k_largest_ops(data_coeff, data_str, 15)
ind20, ops20, vals20 = get_k_largest_ops(data_coeff, data_str, 20)
ind25, ops25, vals25 = get_k_largest_ops(data_coeff, data_str, 25)
ind30, ops30, vals30 = get_k_largest_ops(data_coeff, data_str, 30)
ind40, ops40, vals40 = get_k_largest_ops(data_coeff, data_str, 40)

ops_Hp15 =[Pauli(f'{ops15[i]}') for i in range(len(ops15))]
ops_Hp20 =[Pauli(f'{ops20[i]}') for i in range(len(ops20))]
ops_Hp25 =[Pauli(f'{ops25[i]}') for i in range(len(ops25))]
ops_Hp30 =[Pauli(f'{ops30[i]}') for i in range(len(ops30))]
ops_Hp40 =[Pauli(f'{ops40[i]}') for i in range(len(ops40))]

In [5]:
ev_op_Hp15 = EvolvedOperatorAnsatz(ops_Hp15, reps=1, insert_barriers=True)
ev_op_Hp20 = EvolvedOperatorAnsatz(ops_Hp20, reps=1, insert_barriers=True)
ev_op_Hp25 = EvolvedOperatorAnsatz(ops_Hp25, reps=1, insert_barriers=True)
ev_op_Hp30 = EvolvedOperatorAnsatz(ops_Hp30, reps=1, insert_barriers=True)
ev_op_Hp40 = EvolvedOperatorAnsatz(ops_Hp40, reps=1, insert_barriers=True)

ev_op_Hp15_2f = EvolvedOperatorAnsatz(ops_Hp15, reps=2, insert_barriers=True)
ev_op_Hp20_2f = EvolvedOperatorAnsatz(ops_Hp20, reps=2, insert_barriers=True)
ev_op_Hp25_2f = EvolvedOperatorAnsatz(ops_Hp25, reps=2, insert_barriers=True)
ev_op_Hp30_2f = EvolvedOperatorAnsatz(ops_Hp30, reps=2, insert_barriers=True)
ev_op_Hp40_2f = EvolvedOperatorAnsatz(ops_Hp40, reps=2, insert_barriers=True)

ansatz_list = [ev_op_Hp15, ev_op_Hp20, ev_op_Hp25, ev_op_Hp30, ev_op_Hp40, 
               ev_op_Hp15_2f, ev_op_Hp20_2f, ev_op_Hp25_2f, ev_op_Hp30_2f, ev_op_Hp40_2f]

ansatz_names = ['ev_op_Hp15', 'ev_op_Hp20', 'ev_op_Hp25', 'ev_op_Hp30', 'ev_op_H40',
               'ev_op_Hp15_2f','ev_op_Hp20_2f','ev_op_Hp25_2f', 'ev_op_Hp30_2f', 'ev_op_Hp40_2f']

print(f'number of params: {[ansatz_list[i].num_parameters for i in range(len(ansatz_list))]}')

number of params: [15, 20, 25, 30, 40, 30, 40, 50, 60, 80]


In [5]:
#additional ansatze
ev_op_Hp15_3f = EvolvedOperatorAnsatz(ops_Hp15, reps=3, insert_barriers=True)
ev_op_Hp20_3f = EvolvedOperatorAnsatz(ops_Hp20, reps=3, insert_barriers=True)
ansatz_list = [ev_op_Hp15_3f, ev_op_Hp20_3f]
ansatz_names =['ev_op_Hp15_3f', 'ev_op_Hp20_3f']
print(f'number of params: {[ansatz_list[i].num_parameters for i in range(len(ansatz_list))]}')

number of params: [45, 60]


# VQE individual runs

In [6]:
seed = 170
iterations = 650
algorithm_globals.random_seed = seed

#estimator
noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)
#storing values
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
    
def run_qve_w_specified_optimizer(optimizer, ansatz):
    opt = optimizer(maxiter = iterations)
    vqe = VQE(noiseless_estimator, ansatz, optimizer=opt, callback=store_intermediate_result)
    result = vqe.compute_minimum_eigenvalue(operator=H4q).eigenvalue.real
    print(f"VQE result: {result:.5f}")
    return result

# COBYLA

At step 0, with ev_op_Hp20
VQE result: 3.24109
Length of this optimization 276, time taken = 390.582 

At step 1, with ev_op_Hp30
VQE result: 3.39777
Length of this optimization 398, time taken = 621.303 

At step 2, with ev_op_H40
VQE result: 3.83656
Length of this optimization 550, time taken = 1340.477 

At step 3, with ev_op_Hp20_2f
VQE result: 3.32070
Length of this optimization 444, time taken = 884.188 

At step 4, with ev_op_Hp30_2f
VQE result: 4.13631
Length of this optimization 550, time taken = 1323.844 

At step 5, with ev_op_H40_2f
VQE result: 6.05733
Length of this optimization 550, time taken = 2197.294 

In [7]:
r_cobyla=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')   
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(COBYLA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_cobyla.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))
    

At step 0, with ev_op_Hp15
VQE result: 3.19333
Length of this optimization 204, time taken = 246.569 

At step 1, with ev_op_Hp20
VQE result: 3.19783
Length of this optimization 244, time taken = 327.01 

At step 2, with ev_op_Hp25
VQE result: 3.29968
Length of this optimization 288, time taken = 406.598 

At step 3, with ev_op_Hp30
VQE result: 3.23420
Length of this optimization 438, time taken = 666.928 

At step 4, with ev_op_H40
VQE result: 3.91962
Length of this optimization 531, time taken = 1267.15 

At step 5, with ev_op_Hp15_2f
VQE result: 3.30153
Length of this optimization 368, time taken = 537.76 

At step 6, with ev_op_Hp20_2f
VQE result: 3.54492
Length of this optimization 513, time taken = 958.102 

At step 7, with ev_op_Hp25_2f
VQE result: 3.28540
Length of this optimization 650, time taken = 1342.362 

At step 8, with ev_op_Hp30_2f
VQE result: 3.87371
Length of this optimization 650, time taken = 1459.21 

At step 9, with ev_op_Hp40_2f
VQE result: 13.40265
Length of th

In [9]:
df1 = pd.concat([r_cobyla[i] for i in range(len(r_cobyla))], axis = 1)
df1.to_csv('l4_l05_op_ev_20_30_40_indv_cobyla.csv')

In [7]:
# Additional ansatze
r_cobyla2=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(COBYLA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_cobyla2.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))
    

At step 0, with ev_op_Hp15_3f
VQE result: 3.14718
Length of this optimization 492, time taken = 890.18 

At step 1, with ev_op_Hp20_3f
VQE result: 3.33062
Length of this optimization 650, time taken = 1899.57 



In [8]:
df1b = pd.concat([r_cobyla2[i] for i in range(len(r_cobyla2))], axis = 1)
df1b.to_csv('l4_l05_op_ev_a_20_30_40_indv_cobyla.csv')
df1b

,ev_op_Hp15_3f,ev_op_Hp20_3f
0,9.341976,14.653902
1,9.341976,15.474910
2,9.341976,13.327975
3,9.341976,14.911832
4,10.994969,13.490865
...,...,...
645,NaN,3.352355
646,NaN,3.365936
647,NaN,3.340786
648,NaN,3.376859


# SPSA, 

### 550 iterations

At step 0, with ev_op_Hp20
VQE result: 3.27681
Length of this optimization 1151, time taken = 1979.814 

At step 1, with ev_op_Hp30
VQE result: 3.39317
Length of this optimization 1151, time taken = 2188.474 

At step 2, with ev_op_H40
VQE result: 3.43173
Length of this optimization 1151, time taken = 2886.12 

At step 3, with ev_op_Hp20_2f
VQE result: 3.43783
Length of this optimization 1151, time taken = 2347.521 

At step 4, with ev_op_Hp30_2f
VQE result: 3.27521
Length of this optimization 1151, time taken = 2775.426 

At step 5, with ev_op_H40_2f
VQE result: 8.67427
Length of this optimization 1151, time taken = 4090.701 

In [8]:
r_spsa=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')    
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(SPSA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_spsa.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))

At step 0, with ev_op_Hp15
VQE result: 3.41548
Length of this optimization 1351, time taken = 1870.359 

At step 1, with ev_op_Hp20
VQE result: 3.35646
Length of this optimization 1351, time taken = 2058.685 

At step 2, with ev_op_Hp25
VQE result: 3.42603
Length of this optimization 1351, time taken = 3963.988 

At step 3, with ev_op_Hp30
VQE result: 3.30582
Length of this optimization 1351, time taken = 24525.54 

At step 4, with ev_op_H40
VQE result: 3.38215
Length of this optimization 1351, time taken = 2971.088 

At step 5, with ev_op_Hp15_2f
VQE result: 3.37805
Length of this optimization 1351, time taken = 2178.358 

At step 6, with ev_op_Hp20_2f
VQE result: 3.30794
Length of this optimization 1351, time taken = 2578.238 

At step 7, with ev_op_Hp25_2f
VQE result: 3.33608
Length of this optimization 1351, time taken = 2718.974 

At step 8, with ev_op_Hp30_2f
VQE result: 3.34507
Length of this optimization 1351, time taken = 3009.474 

At step 9, with ev_op_Hp40_2f
VQE result: 8.

In [11]:
df2 = pd.concat([r_spsa[i] for i in range(len(r_spsa))], axis = 1)
df2.to_csv('l4_l05_op_ev_20_30_40_indv_spsa.csv')

In [9]:
r_spsa2=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(SPSA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_spsa2.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))

At step 0, with ev_op_Hp15_3f
VQE result: 3.25784
Length of this optimization 1351, time taken = 3202.713 

At step 1, with ev_op_Hp20_3f
VQE result: 3.35568
Length of this optimization 1351, time taken = 3897.054 



In [10]:
df2b = pd.concat([r_spsa2[i] for i in range(len(r_spsa2))], axis = 1)
df2b.to_csv('l4_l05_op_ev_a_20_30_40_indv_spsa.csv')